**Навигация по уроку**

1. [Telegram бот c ChatGPT на борту. Знакомство с  ChatGPT](https://colab.research.google.com/drive/1tqXt0XNstMb2TeJ8Za9S2xQA6ipjyMPk)
2. [Подготовка данных для обучения chatGPT методом Search-Ask (Практика)](https://colab.research.google.com/drive/1GmRiwmUH8E8KJ4d1g8vDVMGSdfmFFjd5)
3. Домашняя работа

Используя знания из [первой части](https://colab.research.google.com/drive/1tqXt0XNstMb2TeJ8Za9S2xQA6ipjyMPk) урока, а также базы знаний, полученной в [практической части](https://colab.research.google.com/drive/1GmRiwmUH8E8KJ4d1g8vDVMGSdfmFFjd5) урока, создайте телеграм-бот, который будет отвечать на вопросы из вашей базы знаний. Для создания телеграм-бота используйте библиотеку aiogram3. При отправке Telegram-боту команды `/help` он должен возвращать информацию о базе знаний: тематика, число записей в базе знаний, пример запроса к базе. Задание выполните в Блокноте, для этого вам необходимо вспомнить, как запустить асинхронный цикл в Google Colab.




# Алгоритм обучения Search-Ask

С помощью алгоритма Search-Ask мы будем обучать chatGPT отвечать на вопросы о

---

Зимних Олимпийских играх 2022 года. ChatGPT не обучался на этих знаниях и видит их впервые (см. замечание выше, возможно уже "видел"). Наш алгоритм будет выглядеть следующим образом:


1. Подготовка поисковых данных (один раз для каждого документа):
  * *Сбор*: загружаем документы, например, статьи из Википедии о Зимних Олимпийских играх 2022 года;
  * *Фрагментация*: документы разбиваются на короткие, в основном автономные разделы для токенизации;
  * *Токенизация*: каждый раздел преобразуется в последовательность токенов с помощью Openal API;
  * *Хранилище*: последовательности токенов сохраняются (для больших наборов данных рекомендуется использовать векторную базу данных);
2. Search (один раз для каждого запроса):
  * Вопрос пользователя преобразуем в последовательность токенов с помощью OpenAI API;
  * Используя токенизированный пользовательский запрос, *ранжируем* текстовые разделы (фрагментированные документы) по релевантности (схожести) запросу;
3. Ask (один раз за запрос):
  * Вставляем пользовательский вопрос и наиболее релевантные разделы в сообщение для GPT;
  * Получаем и обработаем ответ от GPT.

In [ ]:
# Отключим предупреждения в колабе. Будет меньше лишней информации в выводе
import warnings

warnings.filterwarnings("ignore")


In [ ]:
%pip install openai mwclient mwparserfromhell tiktoken aiogram


/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=2850) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Импортируем библиотеки:
import mwclient  # библиотека для работы с MediaWiki API для загрузки примеров статей Википедии
import mwparserfromhell  # Парсер для MediaWiki
import pandas as pd  # В DataFrame будем хранить базу знаний и результат токинизации базы знаний
from scipy import spatial  # вычисляет сходство векторов
import re  # для вырезания ссылок <ref> из статей Википедии

# import tiktoken  # для подсчета токенов
import tiktoken

# Для вызова OpenAI API
from openai import OpenAI

# Для работы с системными командами
import os
import getpass

# from google.colab import userdata

# Использование ключа API
# os.environ["OPENAI_API_KEY"] = userdata.get('AITUNNEL_API_KEY')

# Адрес сервера
os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

# Выбираем модель
MODEL = "gpt-3.5-turbo"


In [ ]:
# Помещаем OpenAI ключ в переменную окружения с именем 'OPENAI_API_KEY'
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")


In [ ]:
# Создаём объект OpenAI
openai = OpenAI(
    base_url=os.environ.get("OPENAI_BASE_URL"),
    api_key=os.environ.get("OPENAI_API_KEY"),
)


In [ ]:
# Запрос к chatGPT
query = "Когда проходил чемпионат?"
# Отправляем запрос к OpenAI API
response = openai.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Вы отвечаете на вопросы о Чемпионате мира по крикету 2023",
        },
        {"role": "user", "content": query},
    ],
    model=MODEL,
    temperature=0,
)

# Выводим результат
print(response.choices[0].message.content)


Чемпионат мира по крикету 2023 проходил с 9 февраля по 26 марта 2023 года.


## 1. Подготовка поисковых данных

### Сбор данных

Для сбора и подготовки данных нам необходимо:
* Собрать все заголовки статей заданной категории статей Википедии, заходя на первый уровень вложенности статей.
* Разделить каждую статью на разделы (секции) и подразделы. Отбросить менее релевантные разделы, такие как внешние ссылки и сноски.
* Очистить текст, удалив теги ссылок, пробелы и очень короткие секции (разделы).
* Добавить заголовки и подзаголовки к тексту каждого раздела, чтобы помочь chatGPT понять контекст.

#### Сбор заголовков статей из категории

In [ ]:
# поиск страниц Википедии о мотоциклах Хонда

# Задаем категорию и англоязычную версию Википедии для поиска
CATEGORY_TITLE = "Category:Honda_motorcycles"
WIKI_SITE = "en.wikipedia.org"


# Соберем заголовки всех статей
def titles_from_category(
    category: mwclient.listing.Category,  # Задаем типизированный параметр категории статей
    max_depth: int,  # Определяем глубину вложения статей
) -> set[str]:
    """Возвращает набор заголовков страниц в данной категории Википедии и ее подкатегориях."""
    titles = set()  # Используем множество для хранения заголовков статей
    for cm in category.members():  # Перебираем вложенные объекты категории
        if type(cm) == mwclient.page.Page:  # Если объект является страницей
            titles.add(cm.name)  # в хранилище заголовков добавляем имя страницы
        elif (
            isinstance(cm, mwclient.listing.Category) and max_depth > 0
        ):  # Если объект является категорией и глубина вложения не достигла максимальной
            deeper_titles = titles_from_category(
                cm, max_depth=max_depth - 1
            )  # вызываем рекурсивно функцию для подкатегории
            titles.update(
                deeper_titles
            )  # добавление в множество элементов из другого множества
    return titles


# Инициализация объекта MediaWiki
# WIKI_SITE ссылается на англоязычную часть Википедии
site = mwclient.Site(WIKI_SITE)

# Загрузка раздела заданной категории
category_page = site.pages[CATEGORY_TITLE]
# Получение множества всех заголовков категории с вложенностью на один уровень
titles = titles_from_category(category_page, max_depth=1)


print(f"Создано {len(titles)} заголовков статей в категории {CATEGORY_TITLE}.")


Создано 357 заголовков статей в категории Category:Honda_motorcycles.


#### Извлечение секций из документов

Определим секции в документах, которые менее релевантны и их можно отбросить:



In [ ]:
# Задаем секции, которые будут отброшены при парсинге статей
SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


In [ ]:
# Функция возвращает список всех вложенных секций для заданной секции страницы Википедии


def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,  # текущая секция
    parent_titles: list[str],  # Заголовки родителя
    sections_to_ignore: set[str],  # Секции, которые необходимо проигнорировать
) -> list[tuple[list[str], str]]:
    """
    Из раздела Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """

    # Извлекаем заголовки текущей секции
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    # Заголовки Википедии имеют вид: "== Heading =="

    if title.strip("=" + " ") in sections_to_ignore:
        # Если заголовок секции в списке для игнора, то пропускаем его
        return []

    # Объединим заголовки и подзаголовки, чтобы сохранить контекст для chatGPT
    titles = parent_titles + [title]

    # Преобразуем wikicode секции в строку
    full_text = str(section)

    # Выделяем текст секции без заголовка
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        # Если один заголовок, то формируем результирующий список
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        # Формируем результирующий список из текста до первого подзаголовка
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(
                # Вызываем функцию получения вложенных секций для заданной секции
                all_subsections_from_section(subsection, titles, sections_to_ignore)
            )  # Объединяем результирующие списки данной функции и вызываемой
        return results


# Функция возвращает список всех секций страницы, за исключением тех, которые отбрасываем
def all_subsections_from_title(
    title: str,  # Заголовок статьи Википедии, которую парсим
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,  # Секции, которые игнорируем
    site_name: str = WIKI_SITE,  # Ссылка на сайт википедии
) -> list[tuple[list[str], str]]:
    """
    Из заголовка страницы Википедии возвращает список всех вложенных секций.
    Каждый подраздел представляет собой кортеж, где:
      - первый элемент представляет собой список родительских секций, начиная с заголовка страницы
      - второй элемент представляет собой текст секции
    """

    # Инициализация объекта MediaWiki
    # WIKI_SITE ссылается на англоязычную часть Википедии
    site = mwclient.Site(site_name)

    # Запрашиваем страницу по заголовку
    page = site.pages[title]

    # Получаем текстовое представление страницы
    text = page.text()

    # Удобный парсер для MediaWiki
    parsed_text = mwparserfromhell.parse(text)
    # Извлекаем заголовки
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings:  # Если заголовки найдены
        # В качестве резюме берем текст до первого заголовка
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        # Если нет заголовков, то весь текст считаем резюме
        summary_text = str(parsed_text)
    results = [([title], summary_text)]  # Добавляем резюме в результирующий список
    for subsection in parsed_text.get_sections(
        levels=[2]
    ):  # Извлекаем секции 2-го уровня
        results.extend(
            # Вызываем функцию получения вложенных секций для заданной секции
            all_subsections_from_section(subsection, [title], sections_to_ignore)
        )  # Объединяем результирующие списки данной функции и вызываемой
    return results


In [ ]:
# Разбивка статей на секции
# придется немного подождать, так как на парсинг 100 статей требуется около минуты
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Найдено {len(wikipedia_sections)} секций на {len(titles)} страницах")


Найдено 1242 секций на 357 страницах


#### Очистка текста (теги ссылок, пробелы и короткие секции)

In [ ]:
# Очистка текста секции от ссылок <ref>xyz</ref>, начальных и конечных пробелов
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    titles, text = section
    # Удаляем ссылки и другой мусор
    text = re.sub(r"<ref.*?</ref>", "", text)
    text = re.sub(r"(<ref name)|(</ref>)|(<ref>)", "", text)
    text = re.sub(r"/>", "", text)
    text = re.sub(r"{{convert", "", text)
    text = re.sub(r"{{cvt", "", text)
    text = re.sub(r"abbr=on}}", "", text)
    text = re.sub(r"image.*?(jpg|JPG)", "", text)
    text = re.sub(r"refimprove", "", text)
    text = re.sub(r"[\[\]{}]{2,}", " ", text)
    text = re.sub(r"<br", "", text)
    text = re.sub(r"(&nbsp;)", " ", text)
    text = re.sub(r"""class=["|'].*?["|']""", "", text)
    text = re.sub(r"""style=["|'].*?["|']""", "", text)
    text = re.sub(r"""width=["|'].*?["|']""", "", text)
    text = re.sub(r"File:.*?(right|left)", "", text)
    text = re.sub(r"""colspan=["|'].*?["|']""", "", text)
    text = re.sub(r"url.*", "", text)
    # Удаляем пробелы вначале и конце
    text = text.strip()
    return (titles, text)


# Применим функцию очистки ко всем секциям с помощью генератора списков
wikipedia_sections_cleaned = [clean_section(ws) for ws in wikipedia_sections]


# Отфильтруем короткие и пустые секции
def keep_section(section: tuple[list[str], str]) -> bool:
    """Возвращает значение True, если раздел должен быть сохранен, в противном случае значение False."""
    titles, text = section
    # Фильтруем по произвольной длине, можно выбрать и другое значение
    if len(text) < 50:
        return False
    else:
        return True


original_num_sections = len(wikipedia_sections)
wikipedia_sections_cleaned = [
    ws for ws in wikipedia_sections_cleaned if keep_section(ws)
]
print(
    f"Отфильтровано {original_num_sections-len(wikipedia_sections_cleaned)} секций, \
    осталось {len(wikipedia_sections_cleaned)} секций."
)


Отфильтровано 50 секций,     осталось 1192 секций.


Для наглядности выведем 5 секций, заголовок и текст (50 первых символов) каждой секции

In [ ]:
for ws in wikipedia_sections_cleaned[:5]:
    print(ws[0])
    display(ws[1][:100] + "...")
    print()


['Honda CB650 custom']


'|date=January 2017 \n Infobox Motorcycle\n| name = Honda CB650 Custom\n| \n| caption = 1980 Honda CB650 ...'


['Honda CRF150F']


'Infobox Motorcycle\n| name = Honda CRF150F\n| \n| aka =\n| manufacturer =  Honda \n| parent_company =\n| p...'


['Honda CRF150F', '==2003–2005==']


"The 2003 CRF150F was styled after Honda's racing bikes, with tuned suspension and  Convert|156|cc| e..."


['Honda CRF150F', '==2006 redesign==']


"The biggest change for 2006 was to the engine. In other years, the 150F's engine was a smaller versi..."


['Honda CRF150F', '==Specifications==']


'All specifications are manufacturer claimed.\n{|  \n|-\n!\n!  | 2003/2004\n!  | 2005\n!  | 2006/2007\n!  | ...'

### Фрагментация документов

Поскольку chatGPT может одновременно считывать только ограниченный объем токенов, нам необходимо разделить каждый документ на фрагменты (части).

Далее мы рекурсивно разделим длинные разделы на более мелкие. Идеального рецепта разделения текста на разделы не существует.

Однако, можно сделать следующие предположения:

* Более длинные разделы могут быть лучше для вопросов, требующих большего контекста.
* Более длинные разделы могут быть хуже для поиска, поскольку в них может быть больше запутанных тем.
* Более короткие разделы лучше подходят для снижения затрат, которые пропорциональны количеству токенов.
* Более короткие разделы позволяют извлекать больше разделов для запроса, что может помочь с полнотой данных.
* Перекрывающиеся разделы могут помочь предотвратить обрезание ответов по смыслу границами разделов.

Учитывая все выше сказанное, мы будем использовать простой подход и ограничим количество разделов до 1600 токенов в каждом, рекурсивно сокращая вдвое любые слишком длинные разделы. Во избежание сокращения в середине полезных предложений, мы будем разделять секции на части по границам абзацев.

In [ ]:
# Функция подсчета токенов
def num_tokens(text: str, model: str = MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


# Функция разделения строк
def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Разделяет строку надвое с помощью разделителя (delimiter), пытаясь сбалансировать токены с каждой стороны."""

    # Делим строку на части по разделителю, по умолчанию \n - перенос строки
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # разделитель не найден
    elif len(chunks) == 2:
        return chunks  # нет необходимости искать промежуточную точку
    else:
        # Считаем токены
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        # Предварительное разделение по середине числа токенов
        best_diff = halfway
        # В цикле ищем какой из разделителей, будет ближе всего к best_diff
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        # Возвращаем левую и правую часть оптимально разделенной строки
        return [left, right]


# Функция обрезает строку до максимально разрешенного числа токенов
def truncated_string(
    string: str,  # строка
    model: str,  # модель
    max_tokens: int,  # максимальное число разрешенных токенов
    print_warning: bool = True,  # флаг вывода предупреждения
) -> str:
    """Обрезка строки до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    # Обрезаем строку и декодируем обратно
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(
            f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов."
        )
    # Усеченная строка
    return truncated_string


# Функция делит секции статьи на части по максимальному числу токенов
def split_strings_from_subsection(
    subsection: tuple[list[str], str],  # секции
    max_tokens: int = 1000,  # максимальное число токенов
    model: str = MODEL,  # модель
    max_recursion: int = 5,  # максимальное число рекурсий
) -> list[str]:
    """
    Разделяет секции на список из частей секций, в каждой части не более max_tokens.
    Каждая часть представляет собой кортеж родительских заголовков [H1, H2, ...] и текста (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # Если длина соответствует допустимой, то вернет строку
    if num_tokens_in_string <= max_tokens:
        return [string]
    # если в результате рекурсия не удалось разделить строку, то просто усечем ее по числу токенов
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # иначе разделим пополам и выполним рекурсию
    else:
        titles, text = subsection
        for delimiter in [
            "\n\n",
            "\n",
            ". ",
        ]:  # Пробуем использовать разделители от большего к меньшему (разрыв, абзац, точка)
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # если какая-либо половина пуста, повторяем попытку с более простым разделителем
                continue
            else:
                # применим рекурсию на каждой половине
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion
                        - 1,  # уменьшаем максимальное число рекурсий
                    )
                    results.extend(half_strings)
                return results
    # иначе никакого разделения найдено не было, поэтому просто обрезаем строку (должно быть очень редко)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]


In [ ]:
# Делим секции на части
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections_cleaned:
    wikipedia_strings.extend(
        split_strings_from_subsection(section, max_tokens=MAX_TOKENS)
    )

print(
    f"{len(wikipedia_sections_cleaned)} секций Википедии поделены на {len(wikipedia_strings)} строк."
)


1192 секций Википедии поделены на 1245 строк.


In [ ]:
# Напечатаем пример строки
print(wikipedia_strings[1])


Honda CRF150F

Infobox Motorcycle
| name = Honda CRF150F
| 
| aka =
| manufacturer =  Honda 
| parent_company =
| production = 2003–2017
| predecessor =  Honda XR125 
| successor =
| class =  Types of motorcycles#Off-road|Off road 
| engine = |156|cc|,  Radiator (engine cooling)|Air-Cooled ,  Single-cylinder engine|single-cylinder ,  Four-stroke engine|four stroke 

| transmission = 5-speed  Manual transmission|manual 
| suspension =
| brakes = Front: 240mm  Disc brakes|disc  Rear:  Drum brakes|drum 
| tires = Front: 70/100/19 Rear: 90/100/16
| rake_trail =
| wheelbase =
| seat_height = |32.5|in|mm|
| weight =
| fuel_capacity =
| related =  Honda CRF230F   Honda CRF150R   Honda CRF125F 
 
 
The '''Honda CRF150F''' was an  Types of motorcycles#Off-road|off-road   motorcycle  that was first introduced in 2003 as the successor to the  Honda XR series . The 150F was aimed at beginner to intermediate riders, teens or adults. Its main use is for family recreation and easy off-road trails. It

### Токенизация и сохранение результата

Теперь, когда мы разделили нашу базу знаний на более короткие автономные строки, мы можем вычислять эмбединги для каждой строки.

In [ ]:
# Функция отправки chatGPT строки для ее токенизации (вычисления эмбедингов)
def get_embedding(text, model="text-embedding-ada-002"):

    return openai.embeddings.create(input=[text], model=model).data[0].embedding


Поскольку в этом примере используется всего 6 тысяч строк, мы сохраним базу знаний в CSV-файле.

Для больших наборов данных необходимо использовать векторную базу данных, которая будет более производительной.

In [ ]:
df = pd.DataFrame({"text": wikipedia_strings})

df["embedding"] = df.text.apply(lambda x: get_embedding(x))

SAVE_PATH = "./Honda_motorcycles.csv"
# Сохранение результата
df.to_csv(SAVE_PATH, index=False)


INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
/Users/Maverick/Desktop/Миша учеба/Development/Homework/venv/lib/python3.13/site-packages/jupyter_client/session.py:278: RuntimeWarning: coroutine 'main' was never awaited
  return locals()
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https:

In [ ]:
df.head()


,text,embedding
0,Honda CB650 custom\n\n|date=January 2017 \n In...,"[0.007166675757616758, 0.001748416107147932, -..."
1,Honda CRF150F\n\nInfobox Motorcycle\n| name = ...,"[0.010455942712724209, 0.01411976758390665, -0..."
2,Honda CRF150F\n\n==2003–2005==\n\nThe 2003 CRF...,"[-0.0018860958516597748, 0.022216908633708954,..."
3,Honda CRF150F\n\n==2006 redesign==\n\nThe bigg...,"[0.0032024693209677935, 0.026395313441753387, ..."
4,Honda CRF150F\n\n==Specifications==\n\nAll spe...,"[0.007379312999546528, 0.019604353234171867, -..."


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       1245 non-null   object
 1   embedding  1245 non-null   object
dtypes: object(2)
memory usage: 19.6+ KB


## 2. Search (Поиск)

Далее, определим функцию поиска, которая:

* Принимает пользовательский запрос и DataFrame со столбцами `text` и `embedding`;
* Токенизирует пользовательский запрос с помощью OpenAI API (для этого используется токенизатор `text-embedding-ada-002`);
* Использует косинусное расстояние для определения схожести между токенизированым пользовательским запросом и эмбендингами в DataFrame для ранжирования текстов по схожести или релевантности;
* Возвращает два списка:
  * N лучших текстов, ранжированных по релевантности;
  * Их соответствующие оценки релевантности.

In [ ]:
import ast

embeddings_path = "./Honda_motorcycles.csv"

df = pd.read_csv(embeddings_path)
# Конвертируем наши эмбединги из строк в списки
df["embedding"] = df["embedding"].apply(ast.literal_eval)
df.head(10)


,text,embedding
0,Honda CB650 custom\n\n|date=January 2017 \n In...,"[0.007166675757616758, 0.001748416107147932, -..."
1,Honda CRF150F\n\nInfobox Motorcycle\n| name = ...,"[0.010455942712724209, 0.01411976758390665, -0..."
2,Honda CRF150F\n\n==2003–2005==\n\nThe 2003 CRF...,"[-0.0018860958516597748, 0.022216908633708954,..."
3,Honda CRF150F\n\n==2006 redesign==\n\nThe bigg...,"[0.0032024693209677935, 0.026395313441753387, ..."
4,Honda CRF150F\n\n==Specifications==\n\nAll spe...,"[0.007379312999546528, 0.019604353234171867, -..."
5,Honda ST series (minibike)\n\nShort descriptio...,"[-0.011781974695622921, -0.0023785859812051058..."
6,Honda ST series (minibike)\n\n==More specifica...,"[-0.0015647696563974023, -0.000519263790920376..."
7,Honda Sport 90\n\nShort description|Type of mo...,"[-0.0008548901532776654, -0.005027189385145903..."
8,Honda CB400\n\n| Honda CB400F|Honda Dream CB40...,"[-0.0188762079924345, -0.002127999672666192, -..."
9,Honda CB400\n\n== CB440S ==\n\nThe '''Honda CB...,"[-0.021591782569885254, 0.022123927250504494, ..."


In [ ]:
EMBEDDING_MODEL = "text-embedding-ada-002"  # Модель токенизации от OpenAI
# EMBEDDING_MODEL = "text-embedding-3-small"


# Функция поиска
def strings_ranked_by_relatedness(
    query: str,  # пользовательский запрос
    df: pd.DataFrame,  # DataFrame со столбцами text и embedding (база знаний)
    # функция схожести, косинусное расстояние
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,  # выбор лучших n-результатов
) -> tuple[
    list[str], list[float]
]:  # Функция возвращает кортеж двух списков, первый содержит строки, второй - числа с плавающей запятой
    """Возвращает строки и схожести, отсортированные от большего к меньшему"""

    # Отправляем в OpenAI API пользовательский запрос для токенизации
    query_embedding_response = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )

    # Получен токенизированный пользовательский запрос
    query_embedding = query_embedding_response.data[0].embedding

    # Сравниваем пользовательский запрос с каждой токенизированной строкой DataFrame
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    # Сортируем по убыванию схожести полученный список
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Преобразовываем наш список в кортеж из списков
    strings, relatednesses = zip(*strings_and_relatednesses)

    # Возвращаем n лучших результатов
    return strings[:top_n], relatednesses[:top_n]


In [ ]:
query = "What is the engine capacity of the Honda NC series motorcycles?"
strings, relatednesses = strings_ranked_by_relatedness(query, df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)


INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"


relatedness=0.883


'Honda NH series\n\n==Engine sizes==\n\n*NH 50  (49cc)\n*NH 80  (79cc)\n*NH 90  (89cc)\n*NH 100 (96cc)\n*NH 125 (124cc)\n*NH 150 (149cc)\nThere is also a more modern Lead in 100cc, 110cc and 125cc versions.'

relatedness=0.877


'Honda NC700D Integra\n\n==2014 update==\n\nThe NC series was updated for the 2014 model year with an increase in engine capacity to 745 cc (through a 4 mm increase in bore), with power rising to 40.3 kW at 6,250 rpm and torque to 68 Nm at 4,750 rpm. The NC750 Integra also received a new aluminum swing arm, which superseded the box section steel arm of the NC700 Integra . The increased performance was also matched with a software update to the dual clutch transmission and taller gearing that provide improvement to fuel economy. Minor changes to the fairing and seat also provide increased leg room for taller riders.'

...

'Honda NC700 series\n\n==Engine==\n\nThe NC700 series is powered by a single  overhead camshaft  |670|cc|  straight-twin engine|parallel-twin  engine that is tilted 62˚ forward to provide a low centre of gravity, with near uniform weight distribution.\nThe  stroke ratio|undersquare  engine has  programmed fuel injection , separate timing profiles for each cylinder,\nand is tuned to deliver powerful torque in the low- to mid-speed range.=hanlon\nThe engine was designed to deliver a "pleasant throbbing feel" of a  V-twin engine|V-twin  through the use of a  Straight-twin engine#270°|270° crank , which Honda "deliberately designed with a uniaxial primary balancer" even though the primary vibration of the crankshaft could have been balanced perfectly using a biaxial balance shaft.\nThe fuel consumption figure of |3.58|L/100 km| has been attributed to the low number of moving parts in the engine &ndash; the oil pump is driven by the balance shaft, while the camshaft also drives the water pu

relatedness=0.870


"Honda NC700 series\n\nInfobox Motorcycle\n|name             = Honda NC700 series\n||frameless|upright=1.35 \n|caption          = 2012 NC700SA\n|manufacturer     =  Honda|Honda Motor Company \n|production       = 2012—\n|assembly         =  Japan \n|model_year       = \n|class            =  Types of motorcycles#Standard|Standard \n|engine           = Honda RC61 |670|cc|  Overhead camshaft|SOHC   Straight-twin engine|parallel-twin , 4-stroke, 4 valves per cylinder, liquid-cooled\n|bore_stroke      = |73|×|80|mm|\n|compression      = 10.7:1\n|power            = |35|and|40.3|kW|hp| @ 6,250 rpm  Citation needed|date=July 2015 \n|torque           = |83.2|and|87.1|Nm|lbft| @ 4,750 rpm Citation needed|date=July 2015 \n\n|ignition         = Electronic\n|transmission     = 6-speed\n|frame            = Rigid tube steel diamond\n|suspension       = |41|mm| telescopic forks, |120|mm| travel\n|brakes           = |320|mm| single wavy hydraulic disc with 3-piston calipers and sintered metal pads (fro

relatedness=0.865


'Honda NR\n\n==Development==\n\nThe origins of the \'NR\' series of motorcycles lie in Honda\'s return to  Grand Prix motorcycle racing  in the late 1970s following an absence since their highly successful participation in the 1960s. During the absence of Honda, Grand Prix racing came to be dominated by  Two-stroke cycle|two-stroke  machines that could easily attain a higher specific output than a  Four-stroke cycle|four-stroke  equivalent. Honda had long preferred to concentrate on four-stroke development and therefore decided to produce such a machine to challenge their Japanese rivals.\n\nTo achieve this aim Honda could have looked to follow their 1960s practice of increasing the number of cylinders to produce more power.  However, Grand Prix rules at the time required a configuration with maximum of four combustion chambers.  Honda engineers therefore came up with the highly innovative solution of constructing a \'V8\' engine in the form of a four-cylinder.  This was achieved by de

## 3. Ask

С помощью функции поиска, описанной выше, мы теперь можем автоматически извлекать необходимые сведения из DataFrame и вставлять их в сообщения для chatGPT.

Ниже мы определяем функцию ask, которая:

* Принимает запрос пользователя
* Ищет в базе знаний текст релевантный запросу
* Вставляет этот текст в сообщение для GPT
* Отправляет сообщение в GPT
* Возвращает ответ GPT

In [ ]:
# Функция формирования запроса к chatGPT по пользовательскому вопросу и базе знаний
def query_message(
    query: str,  # пользовательский запрос
    df: pd.DataFrame,  # DataFrame со столбцами text и embedding (база знаний)
    token_budget: int,  # ограничение на число отсылаемых токенов в модель
) -> str:
    """Возвращает сообщение для GPT с соответствующими исходными текстами, извлеченными из фрейма данных (базы знаний)."""
    strings, relatednesses = strings_ranked_by_relatedness(
        query, df
    )  # функция ранжирования базы знаний по пользовательскому запросу
    # Шаблон инструкции для chatGPT
    message = 'Use the below articles on the Honda motorcycles to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    # Шаблон для вопроса
    question = f"\n\nQuestion: {query}"

    # Добавляем к сообщению для chatGPT релевантные строки из базы знаний, пока не выйдем за допустимое число токенов
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if num_tokens(message + next_article + question) > token_budget:
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,  # пользовательский запрос
    df: pd.DataFrame = df,  # DataFrame со столбцами text и embedding (база знаний)
    token_budget: int = 4000 - 500,  # ограничение на число отсылаемых токенов в модель
    print_message: bool = False,  # нужно ли выводить сообщение перед отправкой
) -> str:
    """Отвечает на вопрос, используя GPT и базу знаний."""
    # Формируем сообщение к chatGPT (функция выше)
    message = query_message(query, df, token_budget=token_budget)
    # Если параметр True, то выводим сообщение
    if print_message:
        print(message)
    messages = [
        {
            "role": "system",
            "content": "You answer questions about the Honda motorcycles. Answer like you are a Honda fan.",
        },
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=1,  # гиперпараметр степени случайности при генерации текста.
        # Влияет на то, как модель выбирает следующее слово в последовательности.
    )
    response_message = response.choices[0].message.content
    return response_message


Перейдем к самим запросам к нашей базе знаний:

In [ ]:
display(
    ask(
        "What is the engine capacity of the Honda NC series motorcycles?",
        print_message=True,
        token_budget=2000,
    )
)


INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"


Use the below articles on the Honda motorcycles to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."

Wikipedia article section:
"""
Honda NH series

==Engine sizes==

*NH 50  (49cc)
*NH 80  (79cc)
*NH 90  (89cc)
*NH 100 (96cc)
*NH 125 (124cc)
*NH 150 (149cc)
There is also a more modern Lead in 100cc, 110cc and 125cc versions.
"""

Wikipedia article section:
"""
Honda NC700D Integra

==2014 update==

The NC series was updated for the 2014 model year with an increase in engine capacity to 745 cc (through a 4 mm increase in bore), with power rising to 40.3 kW at 6,250 rpm and torque to 68 Nm at 4,750 rpm. The NC750 Integra also received a new aluminum swing arm, which superseded the box section steel arm of the NC700 Integra . The increased performance was also matched with a software update to the dual clutch transmission and taller gearing that provide improvement to fuel economy. Minor changes to the fairing and seat als

INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/chat/completions "HTTP/1.1 200 OK"


'The engine capacity of the Honda NC series motorcycles is 670cc.'

# Создание бота

In [138]:
# Установим библиотеку nest_asyncio
%pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=2850) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import logging
import asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command, CommandStart
from aiogram.types import Message, BotCommand, BotCommandScopeDefault


# Настройка логирования
logging.basicConfig(level=logging.INFO)


In [ ]:
# Использование ключа API
# bot_key = userdata.get("BOT_TOKEN")
bot_key = getpass.getpass("Введите BOT_TOKEN:")
os.environ["BOT_TOKEN"] = bot_key

# Инициализация бота и диспетчера
bot = Bot(token=os.environ.get("BOT_TOKEN"))
dp = Dispatcher()


In [ ]:
# Пример базы знаний
knowledge_base = {
    "тематика": "Мотоциклы Honda",
    "число записей": df.shape[0],
    "пример запроса": "What is the engine capacity of the Honda NC series motorcycles?",
}


# Задаем мену команд бота
async def set_commands():
    commands = [
        BotCommand(command="start", description="Старт"),
        BotCommand(command="help", description="Помощь"),
    ]
    await bot.set_my_commands(commands, BotCommandScopeDefault())


# Хэндлер на команды /start
@dp.message(CommandStart())
async def cmd_start(message: types.Message):
    # Прикрепляем кнопки к сообщению
    await message.reply(
        "Добро пожаловать! Пожалуйста введите свой вопрос о мотоциклах Honda."
    )


# Обработчик команды /help
@dp.message(Command("help"))
async def send_help(message: Message):
    help_text = (
        f"Информация о базе знаний:\n"
        f"Тематика: {knowledge_base['тематика']}\n"
        f"Число записей: {knowledge_base['число записей']}\n"
        f"Пример запроса: {knowledge_base['пример запроса']}"
    )
    await message.answer(help_text)


# Обработчик текстовых сообщений (вопросов)
@dp.message()
async def handle_question(message: types.Message):
    user_question = message.text
    # Передаем вопрос в функцию ask
    response = ask(user_question)
    # Отправляем ответ пользователю
    await message.answer(response)


# Запуск бота
async def main():
    await dp.start_polling(bot)
    await set_commands()


if __name__ == "__main__":
    asyncio.run(main())


INFO:aiogram.dispatcher:Start polling
INFO:aiogram.dispatcher:Run polling for bot @maverick29rus_ai_bot id=8054420741 - 'Honda Expert'
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/chat/completions "HTTP/1.1 200 OK"
INFO:aiogram.event:Update id=298440839 is handled. Duration 11187 ms by bot id=8054420741
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/chat/completions "HTTP/1.1 200 OK"
INFO:aiogram.event:Update id=298440840 is handled. Duration 4353 ms by bot id=8054420741
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/chat/completions "HTTP/1.1 200 OK"
INFO:aiogram.event:Update id=298440841 is handled. Duration 7729 ms by bot id=8054420741
INFO:httpx:HTTP Request: POST https://api.aitunnel.ru/v1/embedd